## Dataloader

## Create index DataFrame

In [1]:
from dataset import load_pdbbind_data_index

index_location = '/home/longyuxi/Documents/mount/pdbbind-dataset/index/INDEX_refined_data.2020'
index = load_pdbbind_data_index(index_location)

## For debug purposes

index = index[:5]
index

,PDB code,resolution,release year,-logKd/Ki,Kd/Ki,reference,ligand name,homology calculated?
0,2r58,2.00,2007,2.0,Kd=10mM,2r58.pdf,MLY,False
1,3c2f,2.35,2008,2.0,Kd=10.1mM,3c2f.pdf,PRP,False
2,3g2y,1.31,2009,2.0,Ki=10mM,3g2y.pdf,GF4,False
3,3pce,2.06,1998,2.0,Ki=10mM,3pce.pdf,3HP,False
4,4qsu,1.90,2014,2.0,Kd=10mM,4qsu.pdf,TDR,False


### Calculating the persistence diagram of each of the coordinates and storing them

In [2]:
from genericpath import isfile
from pathlib import Path

# for each item, calculate the persistence homology of it
output_folder = 'computed_homologies'

# get pdb file for both things
base_folder = Path(index_location).parent.parent / 'refined-set'

# list of commands to be run by GNU parallel
python_interpreter = '/home/longyuxi/miniconda3/envs/donaldlab/bin/python'
homology_calculator_location = '/home/longyuxi/Documents/protein-conformation-topology/calculate_homology.py'
commands = ''

# for any of the files, if they don't exist, calculate homology for it
# (append a line to run the homology calculation file to it) TODO
for idx, row in index.iterrows():
    pdb_name = row['PDB code']
    diagram_output_folder = Path(output_folder) / pdb_name
    diagram_output_folder.mkdir(parents=True, exist_ok=True)

    # protein
    if not (diagram_output_folder / 'protein.pckl').is_file():
        protein_pdb = base_folder / pdb_name / f'{pdb_name}_protein.pdb'
        commands += f'{python_interpreter} {homology_calculator_location} --input {str(protein_pdb)} --output {str(diagram_output_folder / "protein.pckl")}\n'

    # pocket
    if not (diagram_output_folder / 'pocket.pckl').is_file():
        pocket_pdb = base_folder / pdb_name / f'{pdb_name}_pocket.pdb'
        commands += f'{python_interpreter} {homology_calculator_location} --input {str(pocket_pdb)} --output {str(diagram_output_folder / "pocket.pckl")}\n'


    # ligand
    if not (diagram_output_folder / 'ligand.pckl').is_file():
        ligand_mol2 = base_folder / pdb_name / f'{pdb_name}_ligand.mol2'
        commands += f'{python_interpreter} {homology_calculator_location} --input {str(ligand_mol2)} --output {str(diagram_output_folder / "ligand.pckl")}\n'


with open('cmds.tmp', 'w') as cf:
    cf.write(commands)